In [15]:
import numpy as np
import csv

# Game of Life core (Torus/wrapping edges)

def make_grid(rows, cols, p_alive=0.1, seed=None):
    rng = np.random.default_rng(seed)
    return (rng.random((rows, cols)) < p_alive).astype(np.uint8)

def neighbor_count_wrap(G):
    n  = np.roll(G,  1, 0) + np.roll(G, -1, 0)
    n += np.roll(G,  1, 1) + np.roll(G, -1, 1)
    n += np.roll(np.roll(G,  1, 0),  1, 1)
    n += np.roll(np.roll(G,  1, 0), -1, 1)
    n += np.roll(np.roll(G, -1, 0),  1, 1)
    n += np.roll(np.roll(G, -1, 0), -1, 1)
    return n

def step(G):
    N = neighbor_count_wrap(G)
    survive = (G == 1) & ((N == 2) | (N == 3))
    birth   = (G == 0) & (N == 3)
    return (survive | birth).astype(np.uint8)

# SVG output (grid with label)

def grid_to_svg(G, x0, y0, cell, seed, init_live, final_live):
    rows, cols = G.shape
    parts = []

    # decide color based on final vs init
    if init_live == 0:
        color = "black"  # avoid divide-by-zero
    else:
        ratio = final_live / init_live
        if ratio > 1.1:      # significant growth
            color = "green"
        elif ratio < 0.9:    # significant decay
            color = "red"
        else:                # roughly stable
            color = "black"

    # draw live cells in that color
    for r in range(rows):
        for c in range(cols):
            if G[r, c]:
                x = x0 + c * cell
                y = y0 + r * cell
                parts.append(f'<rect x="{x}" y="{y}" width="{cell}" height="{cell}" fill="{color}"/>')

    # box border
    box_w = cols * cell
    box_h = rows * cell
    parts.append(f'<rect x="{x0}" y="{y0}" width="{box_w}" height="{box_h}" '
                 f'fill="none" stroke="#888" stroke-width="1"/>')

    # label
    xc = x0 + cols*cell/2
    yt = y0 + rows*cell + cell*1.5
    label = f"seed={seed} init={init_live} final={final_live}"
    parts.append(f'<text x="{xc}" y="{yt}" font-family="monospace" '
                 f'font-size="{cell*1.2}" text-anchor="middle">{label}</text>')
    return "\n".join(parts)

def save_svg(grids, stats, R, C, cell=5, pad=20, filename="gol_life_tiled.svg"):
    rows, cols = grids[0].shape
    tile_h = rows*cell + int(cell*3)   # room for text
    tile_w = cols*cell
    W = C*tile_w + (C+1)*pad
    H = R*tile_h + (R+1)*pad

    parts = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{W}" height="{H}">']
    parts.append(f'<rect x="0" y="0" width="{W}" height="{H}" fill="white"/>')

    for i, (G, (seed, init_live, final_live)) in enumerate(zip(grids, stats)):
        rr, cc = divmod(i, C)
        x0 = pad + cc*(tile_w + pad)
        y0 = pad + rr*(tile_h + pad)
        parts.append(grid_to_svg(G, x0, y0, cell, seed, init_live, final_live))

    parts.append("</svg>")
    with open(filename, "w") as f:
        f.write("\n".join(parts))
    print(f"SVG saved to {filename}")

if __name__ == "__main__":
    R, C = 10, 10               # tile layout
    gens = 1000                 # steps per run
    rows, cols = 50, 50       # size of each run
    p_alive = 0.1

    grids, stats = [], []
    rng = np.random.default_rng()  # master RNG
    for i in range(R*C):
        seed = int(rng.integers(0, 1_000_000))
        G = make_grid(rows, cols, p_alive, seed)
        init_live = int(G.sum())
        for _ in range(gens):
            G = step(G)
        final_live = int(G.sum())
        grids.append(G)
        stats.append((seed, init_live, final_live))
        
    save_svg(grids, stats, R, C, cell=6, filename="gol_life_tiled.svg")

    csv_file = "gol_life_stats.csv"
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        # headers
        writer.writerow(["run", "seed", "init_live", "final_live", "ratio"])
        # rows
        for i, (seed, init_live, final_live) in enumerate(stats, start=1):
            ratio = final_live / init_live if init_live else 0
            writer.writerow([i, seed, init_live, final_live, f"{ratio:.3f}"])

    print(f"CSV saved to {csv_file}")


SVG saved to gol_life_tiled.svg
CSV saved to gol_life_stats.csv
